In [107]:
class Variable():
    def __init__(self, x, born_fn=None, reqires_grad=True):
        self.x = x
        self.born_fn = born_fn
        self.grad = 0
        self.reqires_grad =reqires_grad
    def backward(self, grad=1):
        if not self.reqires_grad:
            return 
        self.grad += grad
        if self.born_fn is not None:
            self.born_fn.backward(grad)

    def __str__(self):
        return "Variable( x="+str(self.x)[:7] + " requires_grad=" + str(self.reqires_grad) + " grad=" + str(self.grad)[:7] + " born_fn=" + str(self.born_fn) +" )"
    def __repr__(self):
        return self.__str__()

    def __add__(self, other):     return VariableAdd().forward(self, other)
    def __sub__(self, other):     return VariableSub().forward(self, other)
    def __mul__(self, other):     return VariableMul().forward(self, other)    
    def __truediv__(self, other): return VariableDiv().forward(self, other)

class VariableBase():
    def transform(self, x):
        if not isinstance(x, Variable):
            return Variable(x)
        return x

class VariableAdd(VariableBase):
    def forward(self, a, b):
        self.a, self.b = self.transform(a), self.transform(b)
        rez = self.a.x + self.b.x
        reqires_grad = self.a.reqires_grad or self.b.reqires_grad
        return Variable(rez, born_fn=self, reqires_grad=reqires_grad)
    def backward(self, grad=1):
        self.a.backward(grad=grad)
        self.b.backward(grad=grad)

class VariableSub(VariableBase):
    def forward(self, a, b):
        self.a, self.b = self.transform(a), self.transform(b)
        rez = self.a.x - self.b.x
        reqires_grad = self.a.reqires_grad or self.b.reqires_grad
        return Variable(rez, born_fn=self, reqires_grad=reqires_grad)
    def backward(self, grad=1):
        self.a.backward(grad=grad)
        self.b.backward(grad=-grad)

class VariableMul(VariableBase):
    def forward(self, a, b):
        self.a, self.b = self.transform(a), self.transform(b)
        rez = self.a.x * self.b.x
        reqires_grad = self.a.reqires_grad or self.b.reqires_grad
        return Variable(rez, born_fn=self, reqires_grad=reqires_grad)
    def backward(self, grad=1):
        self.a.backward(grad=grad*self.b.x)
        self.b.backward(grad=grad*self.a.x)

class VariableDiv(VariableBase):
    def forward(self, a, b):
        self.a, self.b = self.transform(a), self.transform(b)
        rez = self.a.x / self.b.x
        reqires_grad = self.a.reqires_grad or self.b.reqires_grad
        return Variable(rez, born_fn=self, reqires_grad=reqires_grad)
    def backward(self, grad=1):
        self.a.backward(grad=grad/self.b.x)
        self.b.backward(grad=-grad*self.a.x/(self.b.x**2))



In [108]:
a = Variable(3)
b = Variable(8)
c = Variable(19)
rez = b*2
rez.backward()
a.grad, b.grad, c.grad

(0, 2, 0)

In [109]:
lr = 0.1
x = Variable(10)
for i in range(100):
    rez = (x-34)*(x-8)
    rez.backward()
    x.x -= lr*x.grad
    print(x)
    x.grad = 0
x

Variable( x=12.2 requires_grad=True grad=-22 born_fn=None )
Variable( x=13.9599 requires_grad=True grad=-17.6 born_fn=None )
Variable( x=15.3679 requires_grad=True grad=-14.08 born_fn=None )
Variable( x=16.4944 requires_grad=True grad=-11.264 born_fn=None )
Variable( x=17.3955 requires_grad=True grad=-9.0112 born_fn=None )
Variable( x=18.1164 requires_grad=True grad=-7.2089 born_fn=None )
Variable( x=18.6931 requires_grad=True grad=-5.7671 born_fn=None )
Variable( x=19.1545 requires_grad=True grad=-4.6137 born_fn=None )
Variable( x=19.5236 requires_grad=True grad=-3.6909 born_fn=None )
Variable( x=19.8188 requires_grad=True grad=-2.9527 born_fn=None )
Variable( x=20.0551 requires_grad=True grad=-2.3622 born_fn=None )
Variable( x=20.2440 requires_grad=True grad=-1.8897 born_fn=None )
Variable( x=20.3952 requires_grad=True grad=-1.5118 born_fn=None )
Variable( x=20.5162 requires_grad=True grad=-1.2094 born_fn=None )
Variable( x=20.6129 requires_grad=True grad=-0.9675 born_fn=None )
Varia

Variable( x=20.9999 requires_grad=True grad=0 born_fn=None )